In [20]:
import json
import pandas as pd

In [21]:
train=pd.read_excel('./train_thalia.xlsx')

In [22]:
train

,source,target,des1,des2,label
0,washu./Course/Title,umich./course/prerequisite,washu. Course Title,umich. course prerequisite,0
1,nyu./Course/Days,gatech./Course/Code,nyu. Course Days,gatech. Course Code,0
2,umich./course/@subject,fiu./Course/Prereq,umich. course @subject,fiu. Course Prereq,0
3,hku./Course/CourseSite,harvard./Course/Prereqs,hku. Course CourseSite,harvard. Course Prereqs,0
4,yale./Course/Instructor,gatech./Course/In/#agg,yale. Course Instructor,gatech. Course In #agg,0
...,...,...,...,...,...
749,fsu./Course/Detail/Bldg/Comments,arizona./Course/@Code,fsu. Course Detail Bldg Comments,arizona. Course @Code,0
750,ucla./Course/CourseInfo/Type,rit./Course/CourseNo,ucla. Course CourseInfo Type,rit. Course CourseNo,0
751,fsu./Course/Detail/Bldg/Comments,arizona./Course/Instructor,fsu. Course Detail Bldg Comments,arizona. Course Instructor,0
752,nyu./Course/No,ksu./Course/Details/RefNo,nyu. Course No,ksu. Course Details RefNo,0


In [23]:
match=pd.read_excel('match.xlsx')
match=match.loc[:,~match.columns.str.match("Unnamed")]
match

,source,target,size1,size2,type1,type2,label
0,bu./course/college,arizona./Course/@Code,3,3,string,string,0
1,bu./course/college,arizona./Course/Day,3,4,string,date,0
2,bu./course/college,arizona./Course/Instructor,3,15,string,string,0
3,bu./course/college,arizona./Course/Place,3,12,string,string,0
4,bu./course/college,arizona./Course/Time,3,8,string,time,0
...,...,...,...,...,...,...,...
997,fsu./Course/Section/#agg,arizona./Course/@Code,30,3,integer,string,0
998,fsu./Course/Section/#agg,arizona./Course/Day,30,4,integer,date,0
999,fsu./Course/Section/#agg,arizona./Course/Instructor,30,15,integer,string,0
1000,fsu./Course/Section/#agg,arizona./Course/Place,30,12,integer,string,0


In [24]:
train=pd.merge(match,train,on=['source','target']).drop(columns=['des1','des2'])
train

,source,target,size1,size2,type1,type2,label_x,label_y
0,bu./course/college,arizona./Course/Day,3,4,string,date,0,0
1,bu./course/college,arizona./Course/Instructor,3,15,string,string,0,0
2,bu./course/college,arizona./Course/Place,3,12,string,string,0,0
3,bu./course/college,arizona./Course/Time,3,8,string,time,0,0
4,bu./course/courseInfo/@code,arizona./Course/@Code,7,3,string,string,0,0
...,...,...,...,...,...,...,...,...
749,fsu./Course/Section/#agg,arizona./Course/@Code,30,3,integer,string,0,0
750,fsu./Course/Section/#agg,arizona./Course/Day,30,4,integer,date,0,0
751,fsu./Course/Section/#agg,arizona./Course/Instructor,30,15,integer,string,0,0
752,fsu./Course/Section/#agg,arizona./Course/Place,30,12,integer,string,0,0


In [25]:
type1=pd.get_dummies(train['type1'])
type1=type1.add_suffix('_x')
type2=pd.get_dummies(train['type2'])
type2
type2=type2.add_suffix('_y')
type12=pd.concat([type1,type2],axis=1,join='inner')
type12

,date_x,integer_x,string_x,time_x,date_y,integer_y,string_y,time_y
0,0,0,1,0,1,0,0,0
1,0,0,1,0,0,0,1,0
2,0,0,1,0,0,0,1,0
3,0,0,1,0,0,0,0,1
4,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
749,0,1,0,0,0,0,1,0
750,0,1,0,0,1,0,0,0
751,0,1,0,0,0,0,1,0
752,0,1,0,0,0,0,1,0


In [26]:
train=pd.concat([train,type12],axis=1,join='inner')
train_label=train['label_x']
train=train.drop(columns=['source','target','type2','type1','label_x','label_y'])
train

,size1,size2,date_x,integer_x,string_x,time_x,date_y,integer_y,string_y,time_y
0,3,4,0,0,1,0,1,0,0,0
1,3,15,0,0,1,0,0,0,1,0
2,3,12,0,0,1,0,0,0,1,0
3,3,8,0,0,1,0,0,0,0,1
4,7,3,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
749,30,3,0,1,0,0,0,0,1,0
750,30,4,0,1,0,0,1,0,0,0
751,30,15,0,1,0,0,0,0,1,0
752,30,12,0,1,0,0,0,0,1,0


In [51]:
f=open('./thalia_SMAT/4_train_raw_outputs.json',)
movie_train_raw = json.load(f)
movie_train_raw.keys()
for b in range(3):
    a=movie_train_raw['batch_'+str(b)]
    if not b==0:
        movie_train_raw['batch_0'].extend(a)


In [52]:
len(movie_train_raw['batch_0'])
import torch
import torch.nn as nn
tversion=torch.tensor(movie_train_raw['batch_0'])
m=torch.nn.Softmax(dim=1)
output_movie_train=[i[1] for i in m(tversion).tolist()]
output_movie_train

[0.45127731561660767,
 0.33066314458847046,
 0.4593294560909271,
 0.45413514971733093,
 0.4164014160633087,
 0.4910459816455841,
 0.4789181649684906,
 0.4854496717453003,
 0.48354148864746094,
 0.3719291090965271,
 0.5720481276512146,
 0.4704422354698181,
 0.32429781556129456,
 0.507595956325531,
 0.4082507789134979,
 0.33583909273147583,
 0.35953134298324585,
 0.4311847388744354,
 0.44668158888816833,
 0.4819907546043396,
 0.4202027916908264,
 0.4731219410896301,
 0.4467619061470032,
 0.4427303969860077,
 0.39084702730178833,
 0.42606186866760254,
 0.4163401424884796,
 0.47364237904548645,
 0.33952051401138306,
 0.49104514718055725,
 0.4290059506893158,
 0.4168585538864136,
 0.4632987380027771,
 0.33744147419929504,
 0.4707115590572357,
 0.41349318623542786,
 0.3889934718608856,
 0.4970579445362091,
 0.4563676416873932,
 0.32396337389945984,
 0.30034339427948,
 0.40055015683174133,
 0.32957279682159424,
 0.5372946858406067,
 0.49207472801208496,
 0.47912120819091797,
 0.37664079666137

In [53]:
train['outputscore']=output_movie_train

In [54]:
train

,size1,size2,date_x,integer_x,string_x,time_x,date_y,integer_y,string_y,time_y,outputscore
0,3,4,0,0,1,0,1,0,0,0,0.451277
1,3,15,0,0,1,0,0,0,1,0,0.330663
2,3,12,0,0,1,0,0,0,1,0,0.459329
3,3,8,0,0,1,0,0,0,0,1,0.454135
4,7,3,0,0,1,0,0,0,1,0,0.416401
...,...,...,...,...,...,...,...,...,...,...,...
749,30,3,0,1,0,0,0,0,1,0,0.368496
750,30,4,0,1,0,0,1,0,0,0,0.454735
751,30,15,0,1,0,0,0,0,1,0,0.437906
752,30,12,0,1,0,0,0,0,1,0,0.321556


In [55]:
train_label

0      0
1      0
2      0
3      0
4      0
      ..
749    0
750    0
751    0
752    0
753    0
Name: label_x, Length: 754, dtype: int64

In [56]:
train.to_csv('4_train_encode.csv',index=False)
train_label.to_csv('4_train_label.csv',index=False)

In [ ]:
#val

In [63]:
import json
import pandas as pd

In [64]:
val=pd.read_excel('./val_thalia.xlsx')

In [65]:
val

,source,target,des1,des2,label
0,fsu./Course/Detail/Bldg/Days,arizona./Course/Day,fsu. Course Detail Bldg Days,arizona. Course Day,1
1,hku./Course/Term,harvard./Course/Number,hku. Course Term,harvard. Course Number,0
2,gatech./Course/Description,arizona./Course/Day,gatech. Course Description,arizona. Course Day,0
3,cmu./Course/Time,bu./course/courseInfo/schedule/days,cmu. Course Time,bu. course courseInfo schedule days,0
4,fsu./Course/Detail/Bldg/TimeEnd,arizona./Course/Time,fsu. Course Detail Bldg TimeEnd,arizona. Course Time,1
...,...,...,...,...,...
111,ucla./Course/CourseInfo/Section,rit./Course/#id,ucla. Course CourseInfo Section,rit. Course #id,0
112,gatech./Course/Mode,arizona./Course/@Code,gatech. Course Mode,arizona. Course @Code,0
113,cmu./Course/Day,bu./course/courseInfo/schedule/days,cmu. Course Day,bu. course courseInfo schedule days,0
114,nyu./Course/CallNo,gatech./Course/Room,nyu. Course CallNo,gatech. Course Room,0


In [66]:
match=pd.read_excel('match.xlsx')
match=match.loc[:,~match.columns.str.match("Unnamed")]
match

,source,target,size1,size2,type1,type2,label
0,bu./course/college,arizona./Course/@Code,3,3,string,string,0
1,bu./course/college,arizona./Course/Day,3,4,string,date,0
2,bu./course/college,arizona./Course/Instructor,3,15,string,string,0
3,bu./course/college,arizona./Course/Place,3,12,string,string,0
4,bu./course/college,arizona./Course/Time,3,8,string,time,0
...,...,...,...,...,...,...,...
997,fsu./Course/Section/#agg,arizona./Course/@Code,30,3,integer,string,0
998,fsu./Course/Section/#agg,arizona./Course/Day,30,4,integer,date,0
999,fsu./Course/Section/#agg,arizona./Course/Instructor,30,15,integer,string,0
1000,fsu./Course/Section/#agg,arizona./Course/Place,30,12,integer,string,0


In [67]:
val=pd.merge(match,val,on=['source','target']).drop(columns=['des1','des2'])
val

,source,target,size1,size2,type1,type2,label_x,label_y
0,bu./course/college,arizona./Course/@Code,3,3,string,string,0,0
1,bu./course/courseInfo/schedule/room,arizona./Course/Time,6,8,string,time,0,0
2,bu./course/courseInfo/schedule/time,arizona./Course/@Code,8,3,time,string,0,0
3,bu./course/courseInfo/schedule/time,arizona./Course/Instructor,8,15,time,string,0,0
4,bu./course/courseInfo/title,arizona./Course/Instructor,20,15,string,string,0,0
...,...,...,...,...,...,...,...,...
111,fsu./Course/Detail/Seats,arizona./Course/@Code,45,3,integer,string,0,0
112,fsu./Course/Detail/Seats,arizona./Course/Instructor,45,15,integer,string,0,0
113,fsu./Course/Detail/Seats/#agg,arizona./Course/Instructor,45,15,integer,string,0,0
114,fsu./Course/Detail/Title,arizona./Course/Day,20,4,string,date,0,0


In [68]:
type1=pd.get_dummies(val['type1'])
type1=type1.add_suffix('_x')
type2=pd.get_dummies(val['type2'])
type2
type2=type2.add_suffix('_y')
type12=pd.concat([type1,type2],axis=1,join='inner')
type12

,date_x,integer_x,string_x,time_x,date_y,integer_y,string_y,time_y
0,0,0,1,0,0,0,1,0
1,0,0,1,0,0,0,0,1
2,0,0,0,1,0,0,1,0
3,0,0,0,1,0,0,1,0
4,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
111,0,1,0,0,0,0,1,0
112,0,1,0,0,0,0,1,0
113,0,1,0,0,0,0,1,0
114,0,0,1,0,1,0,0,0


In [69]:
val=pd.concat([val,type12],axis=1,join='inner')
val_label=val['label_x']
val=val.drop(columns=['source','target','type2','type1','label_x','label_y'])
val

,size1,size2,date_x,integer_x,string_x,time_x,date_y,integer_y,string_y,time_y
0,3,3,0,0,1,0,0,0,1,0
1,6,8,0,0,1,0,0,0,0,1
2,8,3,0,0,0,1,0,0,1,0
3,8,15,0,0,0,1,0,0,1,0
4,20,15,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
111,45,3,0,1,0,0,0,0,1,0
112,45,15,0,1,0,0,0,0,1,0
113,45,15,0,1,0,0,0,0,1,0
114,20,4,0,0,1,0,1,0,0,0


In [94]:
f=open('./thalia_SMAT/4_val_raw_outputs.json',)
movie_train_raw = json.load(f)
movie_train_raw.keys()
for b in range(1):
    a=movie_train_raw['batch_'+str(b)]
    if not b==0:
        movie_train_raw['batch_0'].extend(a)


In [95]:
len(movie_train_raw['batch_0'])
import torch
import torch.nn as nn
tversion=torch.tensor(movie_train_raw['batch_0'])
m=torch.nn.Softmax(dim=1)
output_movie_train=[i[1] for i in m(tversion).tolist()]
output_movie_train

[0.4623488485813141,
 0.46351343393325806,
 0.4769400954246521,
 0.3755214810371399,
 0.48071855306625366,
 0.29731065034866333,
 0.43561938405036926,
 0.3250267803668976,
 0.48883283138275146,
 0.4773642420768738,
 0.3440663516521454,
 0.3238012492656708,
 0.30503255128860474,
 0.3290374279022217,
 0.4759880304336548,
 0.4504464566707611,
 0.3286183476448059,
 0.455157071352005,
 0.3173369765281677,
 0.3527626693248749,
 0.42683374881744385,
 0.48774296045303345,
 0.4192444086074829,
 0.40783268213272095,
 0.5223124027252197,
 0.4463340938091278,
 0.470796138048172,
 0.48234471678733826,
 0.4396290183067322,
 0.5347381234169006,
 0.49586090445518494,
 0.2944045066833496,
 0.4384286403656006,
 0.5262641906738281,
 0.4519351124763489,
 0.4619879126548767,
 0.38470572233200073,
 0.32852980494499207,
 0.47137323021888733,
 0.3552657663822174,
 0.36690524220466614,
 0.28883635997772217,
 0.48446395993232727,
 0.38723042607307434,
 0.4332224130630493,
 0.34424495697021484,
 0.39245057106018

In [96]:
val['outputscore']=output_movie_train

In [97]:
val

,size1,size2,date_x,integer_x,string_x,time_x,date_y,integer_y,string_y,time_y,outputscore
0,3,3,0,0,1,0,0,0,1,0,0.462349
1,6,8,0,0,1,0,0,0,0,1,0.463513
2,8,3,0,0,0,1,0,0,1,0,0.476940
3,8,15,0,0,0,1,0,0,1,0,0.375521
4,20,15,0,0,1,0,0,0,1,0,0.480719
...,...,...,...,...,...,...,...,...,...,...,...
111,45,3,0,1,0,0,0,0,1,0,0.403217
112,45,15,0,1,0,0,0,0,1,0,0.376285
113,45,15,0,1,0,0,0,0,1,0,0.374014
114,20,4,0,0,1,0,1,0,0,0,0.316097


In [98]:
val_label

0      0
1      0
2      0
3      0
4      0
      ..
111    0
112    0
113    0
114    0
115    0
Name: label_x, Length: 116, dtype: int64

In [99]:
val.to_csv('4_val_encode.csv',index=False)
val_label.to_csv('4_val_label.csv',index=False)

In [ ]:
#test

In [100]:
import json
import pandas as pd

In [101]:
val=pd.read_excel('./test_thalia.xlsx')

In [102]:
val

,source,target,des1,des2,label
0,nyu./Course/Room,gatech./Course/Code,nyu. Course Room,gatech. Course Code,0
1,nyu./Course/Time,ksu./Course/Details/Room,nyu. Course Time,ksu. Course Details Room,0
2,nyu./Course/No,ksu./Course/Title,nyu. Course No,ksu. Course Title,0
3,ucla./Course/CourseInfo/Room,rit./Course/Section/Instructor,ucla. Course CourseInfo Room,rit. Course Section Instructor,0
4,ucla./Course/CourseInfo/Section,rit./Course/Section/Instructor,ucla. Course CourseInfo Section,rit. Course Section Instructor,0
...,...,...,...,...,...
127,washu./Course/Instructor,fiu./Course/Prereq,washu. Course Instructor,fiu. Course Prereq,0
128,nyu./Course/Days,gatech./Course/Description,nyu. Course Days,gatech. Course Description,0
129,yale./Course/Instructor,gatech./Course/Code/#agg,yale. Course Instructor,gatech. Course Code #agg,0
130,toronto./course/coursewebsite,metu./Course/@Code,toronto. course coursewebsite,metu. Course @Code,0


In [103]:
match=pd.read_excel('match.xlsx')
match=match.loc[:,~match.columns.str.match("Unnamed")]
match

,source,target,size1,size2,type1,type2,label
0,bu./course/college,arizona./Course/@Code,3,3,string,string,0
1,bu./course/college,arizona./Course/Day,3,4,string,date,0
2,bu./course/college,arizona./Course/Instructor,3,15,string,string,0
3,bu./course/college,arizona./Course/Place,3,12,string,string,0
4,bu./course/college,arizona./Course/Time,3,8,string,time,0
...,...,...,...,...,...,...,...
997,fsu./Course/Section/#agg,arizona./Course/@Code,30,3,integer,string,0
998,fsu./Course/Section/#agg,arizona./Course/Day,30,4,integer,date,0
999,fsu./Course/Section/#agg,arizona./Course/Instructor,30,15,integer,string,0
1000,fsu./Course/Section/#agg,arizona./Course/Place,30,12,integer,string,0


In [104]:
val=pd.merge(match,val,on=['source','target']).drop(columns=['des1','des2'])
val

,source,target,size1,size2,type1,type2,label_x,label_y
0,bu./course/courseInfo/instructor,arizona./Course/@Code,10,3,string,string,0,0
1,bu./course/courseInfo/instructor,arizona./Course/Day,10,4,string,date,0,0
2,bu./course/courseInfo/schedule/days,arizona./Course/Place,4,12,date,string,0,0
3,bu./course/courseInfo/schedule/room,arizona./Course/@Code,6,3,string,string,0,0
4,bu./course/courseInfo/schedule/room,arizona./Course/Instructor,6,15,string,string,0,0
...,...,...,...,...,...,...,...,...
127,fsu./Course/Detail/SeatsOpen,arizona./Course/Day,35,4,integer,date,0,0
128,fsu./Course/Detail/SeatsOpen,arizona./Course/Place,35,12,integer,string,0,0
129,fsu./Course/Detail/SeatsOpen/#agg,arizona./Course/Time,35,8,integer,time,0,0
130,fsu./Course/ReferenceNo,arizona./Course/@Code,4,3,string,string,0,0


In [105]:
type1=pd.get_dummies(val['type1'])
type1=type1.add_suffix('_x')
type2=pd.get_dummies(val['type2'])
type2
type2=type2.add_suffix('_y')
type12=pd.concat([type1,type2],axis=1,join='inner')
type12

,date_x,integer_x,string_x,time_x,date_y,integer_y,string_y,time_y
0,0,0,1,0,0,0,1,0
1,0,0,1,0,1,0,0,0
2,1,0,0,0,0,0,1,0
3,0,0,1,0,0,0,1,0
4,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...
127,0,1,0,0,1,0,0,0
128,0,1,0,0,0,0,1,0
129,0,1,0,0,0,0,0,1
130,0,0,1,0,0,0,1,0


In [106]:
val=pd.concat([val,type12],axis=1,join='inner')
val_label=val['label_x']
val=val.drop(columns=['source','target','type2','type1','label_x','label_y'])
val

,size1,size2,date_x,integer_x,string_x,time_x,date_y,integer_y,string_y,time_y
0,10,3,0,0,1,0,0,0,1,0
1,10,4,0,0,1,0,1,0,0,0
2,4,12,1,0,0,0,0,0,1,0
3,6,3,0,0,1,0,0,0,1,0
4,6,15,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
127,35,4,0,1,0,0,1,0,0,0
128,35,12,0,1,0,0,0,0,1,0
129,35,8,0,1,0,0,0,0,0,1
130,4,3,0,0,1,0,0,0,1,0


In [131]:
f=open('./thalia_SMAT/4_test_raw_outputs.json',)
movie_train_raw = json.load(f)
movie_train_raw.keys()
for b in range(1):
    a=movie_train_raw['batch_'+str(b)]
    if not b==0:
        movie_train_raw['batch_0'].extend(a)


In [132]:
len(movie_train_raw['batch_0'])
import torch
import torch.nn as nn
tversion=torch.tensor(movie_train_raw['batch_0'])
m=torch.nn.Softmax(dim=1)
output_movie_train=[i[1] for i in m(tversion).tolist()]
output_movie_train

[0.31432896852493286,
 0.3302477300167084,
 0.3930324912071228,
 0.46527618169784546,
 0.47138527035713196,
 0.44164761900901794,
 0.2974220812320709,
 0.47092780470848083,
 0.5363446474075317,
 0.3809254467487335,
 0.47677314281463623,
 0.437847763299942,
 0.3253156244754791,
 0.46526598930358887,
 0.41728177666664124,
 0.47593021392822266,
 0.3565618693828583,
 0.4122644066810608,
 0.45783284306526184,
 0.44119030237197876,
 0.4471026360988617,
 0.31158682703971863,
 0.4217336177825928,
 0.46489036083221436,
 0.31842246651649475,
 0.43086567521095276,
 0.43997350335121155,
 0.35912197828292847,
 0.46108853816986084,
 0.4509117305278778,
 0.3387758731842041,
 0.3444482088088989,
 0.4574289917945862,
 0.2919614017009735,
 0.46034616231918335,
 0.48050859570503235,
 0.4242519736289978,
 0.3924056887626648,
 0.3736828565597534,
 0.3185330927371979,
 0.32596680521965027,
 0.3484002947807312,
 0.5087298154830933,
 0.4958277940750122,
 0.44591447710990906,
 0.4139067530632019,
 0.4522644877

In [133]:
val['outputscore']=output_movie_train

In [134]:
val

,size1,size2,date_x,integer_x,string_x,time_x,date_y,integer_y,string_y,time_y,outputscore
0,10,3,0,0,1,0,0,0,1,0,0.314329
1,10,4,0,0,1,0,1,0,0,0,0.330248
2,4,12,1,0,0,0,0,0,1,0,0.393032
3,6,3,0,0,1,0,0,0,1,0,0.465276
4,6,15,0,0,1,0,0,0,1,0,0.471385
...,...,...,...,...,...,...,...,...,...,...,...
127,35,4,0,1,0,0,1,0,0,0,0.497624
128,35,12,0,1,0,0,0,0,1,0,0.341933
129,35,8,0,1,0,0,0,0,0,1,0.410184
130,4,3,0,0,1,0,0,0,1,0,0.402983


In [135]:
val_label

0      0
1      0
2      0
3      0
4      0
      ..
127    0
128    0
129    0
130    0
131    0
Name: label_x, Length: 132, dtype: int64

In [136]:
val.to_csv('4_test_encode.csv',index=False)
val_label.to_csv('4_test_label.csv',index=False)